****This is the SVM Testing python notebook, everything needed to test results is contained within this notebook****


In [3]:
from src.segment_full_slic import run_full_slic

# SLIC Settings
num_superpixels = 1024
m = 1
max_iterations = 10
threshold = 20

run_full_slic("src/data/raw/test-images/leftImg8bit/train", num_superpixels, m, max_iterations, threshold)

Processing dataset using SLIC
Step 1: Loading images locations
Loaded folder of images: src/data/raw/test-images/leftImg8bit/train
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/aachen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/bochum
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/bremen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/cologne
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/darmstadt
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/dusseldorf
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-imag

KeyboardInterrupt: 

In [5]:
import cv2
from labeling.image_load_util import load_folder_image, get_full_path_from_root
from utils.serialize import read_clusters, read_matrix
import os

# This cell takes the slic results and loads them for use
matrix_file = open("data/slic/full_slic_matrix.bin", "rb")
cluster_file = open("data/slic/full_slic_cluster.bin", "rb")
print("Files loaded")

# Load the matrix and cluster
matrix = read_matrix(matrix_file) # Matrix size image, pixel corresponds to cluster
cluster = read_clusters(cluster_file) # Cluster ID, avg colour and centroid.

image_directory = 'src/data/raw/test-images/leftImg8bit/train'

image_filenames =  load_folder_image(image_directory)

id_directory = 'src/data/raw/test-images/gtFine/train' # This is the ground truth labels
search_string = 'label'  # Replace this with the substring you want to match
id_filenames = []

all_filenames = load_folder_image(id_directory)
for id in all_filenames:
    if search_string in str(id):
        id_filenames.append(id)


matrix_file.close()
cluster_file.close()

Files loaded
Loaded folder of images: src/data/raw/test-images/leftImg8bit/train
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/aachen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/bochum
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/bremen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/cologne
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/darmstadt
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/dusseldorf
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/leftImg8bit/train/erfurt
Loaded folder of image

In [6]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed

def extract_statistical_features_from_pixels(pixels, num_channels=3):
    """
    Compute the channel mean, std, median, min, and max of each pixel across all channels.
    """
    if pixels.size == 0:
        return np.zeros(num_channels * 5)
    means = np.mean(pixels, axis=0)
    stds = np.std(pixels, axis=0)
    medians = np.median(pixels, axis=0)
    mins = np.min(pixels, axis=0)
    maxs = np.max(pixels, axis=0)
    return np.concatenate([means, stds, medians, mins, maxs])

def lbp_vectorized(image_gray):
    """
    Compute the Local Binary Pattern (LBP) for a grayscale image using vectorized operations.
    """
    center = image_gray.astype(np.uint8)
    lbp = np.zeros_like(center, dtype=np.uint8)

    lbp |= ((np.roll(np.roll(center, 1, axis=0), 1, axis=1) > center).astype(np.uint8)) << 7
    lbp |= ((np.roll(center, 1, axis=0) > center).astype(np.uint8)) << 6
    lbp |= ((np.roll(np.roll(center, 1, axis=0), -1, axis=1) > center).astype(np.uint8)) << 5
    lbp |= ((np.roll(center, -1, axis=1) > center).astype(np.uint8)) << 4
    lbp |= ((np.roll(np.roll(center, -1, axis=0), -1, axis=1) > center).astype(np.uint8)) << 3
    lbp |= ((np.roll(center, -1, axis=0) > center).astype(np.uint8)) << 2
    lbp |= ((np.roll(np.roll(center, -1, axis=0), 1, axis=1) > center).astype(np.uint8)) << 1
    lbp |= ((np.roll(center, 1, axis=1) > center).astype(np.uint8))
    return lbp

def compute_gabor_responses(gray_image, orientations=4, ksize=31, sigma=4.0, lambd=10.0, gamma=0.5, psi=0):
    """
    Precompute the Gabor filter responses for the entire grayscale image.
    Returns an array of shape (orientations, height, width).
    """
    responses = []
    for theta in np.linspace(0, np.pi, orientations, endpoint=False):
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(gray_image, cv2.CV_32F, kernel)
        responses.append(filtered)
    return np.array(responses)

def prepare_training_data(cluster_matrices, ground_truth_matrices, slic_matrices, images,
                          orientations=4, ksize=31, sigma=4.0, lambd=10.0, gamma=0.5, psi=0):
    """
    Computes x_train and y_train for the entire training set.
    """
    results = []
    for j, (clusters, gt, image, slic) in enumerate(zip(cluster_matrices, ground_truth_matrices, images, slic_matrices)):
        # Precompute indices for each segment.
        segment_ids = np.unique(slic)
        indices_dict = {sid: np.where(slic == sid) for sid in segment_ids}

        # Compute grayscale, LBP image, and Gabor responses once per image.
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp_image = lbp_vectorized(gray)
        gabor_responses = compute_gabor_responses(gray, orientations, ksize, sigma, lambd, gamma, psi)

        for cluster in clusters:
            cid = cluster.cid
            mask_indices = indices_dict.get(cid + 1, (np.array([], dtype=int), np.array([], dtype=int)))
            masked_pixels = image[mask_indices]
            stats = extract_statistical_features_from_pixels(masked_pixels, num_channels=image.shape[-1])

            # LBP histogram.
            lbp_values = lbp_image[mask_indices]
            if lbp_values.size == 0:
                texture_hist = np.zeros(256, dtype=float)
            else:
                texture_hist, _ = np.histogram(lbp_values, bins=256, range=(0, 256), density=True)

            # Compute Gabor features using precomputed responses.
            gabor_feat = []
            for resp in gabor_responses:
                values = resp[mask_indices]
                if values.size == 0:
                    mean_response, std_response = 0.0, 0.0
                else:
                    mean_response = np.mean(values)
                    std_response = np.std(values)
                gabor_feat.extend([mean_response, std_response])
            gabor_feat = np.array(gabor_feat)

            # Combine features: color/centroid, statistical, LBP, and Gabor.
            feature_vector = np.concatenate((
                [cluster.l, cluster.a, cluster.b, cluster.x, cluster.y],
                stats,
                texture_hist,
                gabor_feat
            ))

            # Compute dominant ground truth label.
            gt_pixels = gt[mask_indices]
            if gt_pixels.size == 0:
                dominant_label = -1
            else:
                labels, counts = np.unique(gt_pixels, return_counts=True)
                dominant_label = labels[np.argmax(counts)]

            results.append((feature_vector, dominant_label))
        print("Processed image:", j)
    X_train, y_train = zip(*results)
    return np.array(X_train), np.array(y_train)

def train_svm(X_train, y_train):
    pipeline = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10, gamma='scale', probability=True))
    pipeline.fit(X_train, y_train)
    return pipeline

def predict_cluster_labels(cluster_matrix, model, segments, X_train):
    predictions = model.predict(X_train)
    segmentation_result = np.zeros(segments.shape, dtype=np.int32)
    for cluster, label in zip(cluster_matrix, predictions):
        segmentation_result[segments == (cluster.cid + 1)] = label
    return segmentation_result

In [ ]:
# THIS IS FOR FINDING BEST SVM OPTIMIZATION - TAKES FOREVER

from sklearn.model_selection import GridSearchCV

def train_svm_grid_search(X_train, y_train):
    # Define a grid of hyperparameters to search.
    param_grid = {
        'svc__C': [0.1, 1, 10, 100],
        'svc__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
        'svc__kernel': ['rbf', 'linear']
    }

    # Create a pipeline with standardization and SVM.
    pipeline = make_pipeline(StandardScaler(), SVC(probability=True))

    # Setup grid search with cross-validation.
    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train) rid.best_params_)
    print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
    return grid.best_estimator_

# Use the grid search version to train your model.
best_model = train_svm_grid_search(x_train1[0:5000], y_train1[0:5000])

In [ ]:
id_directory = 'src/data/raw/test-images/gtFine/train'
id_directory_photos = 'src/data/raw/test-images/leftImg8bit/train'
# search_string = 'instance'  # Replace this with the substring you want to match
search_string = 'label'  # Replace this with the substring you want to match
id_filenames = []
id_filenames_photos = []

all_filenames = load_folder_image(id_directory)
for id in all_filenames:
    if search_string in str(id):
        id_filenames.append(id)

all_filenames = load_folder_image(id_directory_photos)
for id in all_filenames:
    id_filenames_photos.append(cv2.imread(id, cv2.IMREAD_COLOR))

ground_truth_matrices = []
for id_filename in id_filenames:
    ground_truth_matrices.append(cv2.imread(id_filename, cv2.IMREAD_GRAYSCALE))

Loaded folder of images: src/data/raw/test-images/gtFine/train
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/aachen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/bochum
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/bremen
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/cologne
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/darmstadt
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/dusseldorf
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentation/src/data/raw/test-images/gtFine/train/erfurt
Loaded folder of images: /home/aidan/Documents/Projects/SemanticSegmentatio

In [ ]:
import pickle

matrix_file = open("data/slic/full_slic_matrix.bin", "rb")
cluster_file = open("data/slic/full_slic_cluster.bin", "rb")
appended_gt = []
matrix_cell = []
cluster_cell = []
image_cell = []


for i in range(0,100):
    cluster_cell.append(read_clusters(cluster_file))
    matrix_cell.append(read_matrix(matrix_file))
    appended_gt.append(ground_truth_matrices[i])
    image_cell.append(id_filenames_photos[i])

x_train1, y_train1 = prepare_training_data(cluster_cell, appended_gt, matrix_cell,image_cell)
matrix_file.close()
cluster_file.close()

In [ ]:
# print(x_train)
print(np.shape(y_train1))
print(np.shape(x_train1))

In [ ]:
model = train_svm(x_train1[0:20000], y_train1[0:20000]) # 10000 has 79% acc at 512 slic

In [ ]:
test_image = 9 # 9 at 77.5%, 11, 15,16, 3, 18, 22
cluster_number = 1024
predicted_labels = predict_cluster_labels(cluster_cell[test_image], model, matrix_cell[test_image], x_train1[(cluster_number*test_image)-1:(cluster_number*(1+test_image))])

In [ ]:
def calculate_matching_percentage(matrix1, matrix2):
    # Ensure both matrices have the same shape
    if np.shape(matrix1) != np.shape(matrix2):
        raise ValueError("Matrices must have the same shape")

    # Compare the matrices element-wise
    matching_elements = (matrix1 == matrix2)

    # Calculate the percentage of matching values
    matching_percentage = np.sum(matching_elements) / matching_elements.size * 100

    return matching_percentage

In [ ]:
def label_consolidate(matrix):
    matrix = np.where((7 <= matrix) & (matrix <= 10), 7, matrix)
    matrix = np.where((11 <= matrix) & (matrix <= 16), 11, matrix)
    matrix = np.where((17 <= matrix) & (matrix <= 20), 17, matrix)
    matrix = np.where((21 <= matrix) & (matrix <= 22), 21, matrix)
    matrix = np.where((24 <= matrix) & (matrix <= 25), 24, matrix)
    matrix = np.where((matrix < 0) | (matrix > 25), 33, matrix)
    return matrix

def label_consolidate_colour(matrix):
    rgb_image = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Define masks
    masks = [
        ((0 <= matrix) & (matrix <= 6),       [111, 74, 0]),
        ((7 <= matrix) & (matrix <= 10),      [52, 152, 219 ]),
        ((11 <= matrix) & (matrix <= 16),     [22, 160, 133 ]),
        ((17 <= matrix) & (matrix <= 20),     [243, 156, 18 ]),
        ((21 <= matrix) & (matrix <= 22),     [249, 231, 159 ]),
        ((24 <= matrix) & (matrix <= 25),     [165, 105, 189 ]),
        ((matrix < 0) | (matrix > 25),        [169, 50, 38])
    ]

    # Assign colors
    for mask, color in masks:
        rgb_image[mask] = color

    return rgb_image

In [ ]:
# Less Restricted labels
predicted_image = label_consolidate(predicted_labels)
gt_image = label_consolidate(appended_gt[test_image])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.patches import Patch

bounding_image = label_consolidate_colour(predicted_labels)
image = id_filenames_photos[test_image]

plt.figure(figsize=(8, 8))
plt.imshow(image)  # Display original image
plt.show()
plt.figure(figsize=(8, 8))
# Define custom legend
legend_elements = [
    Patch(facecolor=np.array([111, 74, 0])/255,      label='Unlabelled'),
    Patch(facecolor=np.array([52, 152, 219])/255,    label='Road/Ground'),
    Patch(facecolor=np.array([22, 160, 133])/255,    label='Building/Wall'),
    Patch(facecolor=np.array([243, 156, 18])/255,    label='Pole/Traffic Light'),
    Patch(facecolor=np.array([249, 231, 159])/255,   label='Vegetation'),
    Patch(facecolor=np.array([165, 105, 189])/255,   label='Person'),
    Patch(facecolor=np.array([169, 50, 38])/255,     label='Vehicle')
]

# Display the legend
plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.axis('off')  # Hide axis for cleaner visualization
plt.tight_layout()
plt.imshow(bounding_image, alpha=0.5)  # Overlay bounding image with 50% transparency
plt.axis('off')  # Optional: Hide axis ticks
plt.show()

In [ ]:
percentage = calculate_matching_percentage(predicted_image, gt_image)

print(f"Matching percentage: {percentage:.2f}%")

In [ ]:
from utils.segmentation_utils import segmented_to_color

# Display Segmented image with color
color_segmented_matrix = segmented_to_color(matrix_cell[test_image], cluster_cell[test_image])
plt.imshow(color_segmented_matrix)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(predicted_image, cmap='gray')
plt.colorbar()
plt.imshow(gt_image, cmap='gray')
plt.colorbar()

In [ ]:
avg_accuracy = 0
max_iter = 100

# Define custom legend
legend_elements = [
    Patch(facecolor=np.array([111, 74, 0])/255,      label='Unlabelled'),
    Patch(facecolor=np.array([52, 152, 219])/255,    label='Road/Ground'),
    Patch(facecolor=np.array([22, 160, 133])/255,    label='Building/Wall'),
    Patch(facecolor=np.array([243, 156, 18])/255,    label='Pole/Traffic Light'),
    Patch(facecolor=np.array([249, 231, 159])/255,   label='Vegetation'),
    Patch(facecolor=np.array([165, 105, 189])/255,   label='Person'),
    Patch(facecolor=np.array([169, 50, 38])/255,     label='Vehicle')
]

for i in range(0,max_iter):
    predicted_labels = predict_cluster_labels(cluster_cell[i], model, matrix_cell[i], x_train1[(cluster_number*i):(cluster_number*(1+i))])
    predicted_image = label_consolidate(predicted_labels)
    bounding_image = label_consolidate_colour(predicted_labels)
    gt_image_test = label_consolidate(appended_gt[i])
    percentage = calculate_matching_percentage(predicted_image, gt_image_test)
    avg_accuracy += percentage
    print(f"At iteration {i} matching percentage: {percentage:.2f}%")
    plt.figure(figsize=(8, 8))
    plt.imshow(id_filenames_photos[i])
    plt.show()
    plt.close()
    plt.figure(figsize=(8, 8))
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.axis('off')  # Hide axis for cleaner visualization
    plt.tight_layout()
    plt.imshow(bounding_image)
    plt.show()
    plt.close()

avg_accuracy = avg_accuracy/max_iter
print(f"Average accuracy: {avg_accuracy:.2f}%")

In [ ]:
print(avg_accuracy)

In [ ]:
avg_accuracy = 0
max_iter = 70
start_point = 60

# Define custom legend
legend_elements = [
    Patch(facecolor=np.array([111, 74, 0])/255,      label='Unlabelled'),
    Patch(facecolor=np.array([52, 152, 219])/255,    label='Road/Ground'),
    Patch(facecolor=np.array([22, 160, 133])/255,    label='Building/Wall'),
    Patch(facecolor=np.array([243, 156, 18])/255,    label='Pole/Traffic Light'),
    Patch(facecolor=np.array([249, 231, 159])/255,   label='Vegetation'),
    Patch(facecolor=np.array([165, 105, 189])/255,   label='Person'),
    Patch(facecolor=np.array([169, 50, 38])/255,     label='Vehicle')
]

for i in range(start_point, max_iter):
    predicted_labels = predict_cluster_labels(cluster_cell[i], model, matrix_cell[i], x_train1[(cluster_number*i):(cluster_number*(1+i))])
    predicted_image = label_consolidate(predicted_labels)
    bounding_image = label_consolidate_colour(predicted_labels)
    gt_image_test = label_consolidate(appended_gt[i])
    percentage = calculate_matching_percentage(predicted_image, gt_image_test)
    avg_accuracy += percentage
    print(f"At iteration {i} matching percentage: {percentage:.2f}%")
    plt.figure(figsize=(8, 8))
    plt.imshow(id_filenames_photos[i])
    plt.show()
    plt.close()
    plt.figure(figsize=(8, 8))
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.axis('off')  # Hide axis for cleaner visualization
    plt.tight_layout()
    plt.imshow(bounding_image)
    plt.show()
    plt.close()

avg_accuracy = avg_accuracy/(max_iter-start_point)
print(f"Average accuracy: {avg_accuracy:.2f}%")